# Lab 5: Music classification

SMC, 2016

**This script can be used to extract descriptors from recordings in multiple classes into ARFF file for further processing in WEKA.** It uses descriptors from MIREDU.

Directory structure must be the following:
* *This .ipynb file*
* sonic-annotator (binary version of extractor)
* miredu_all.n3
* miredu_rms.n3
* recordings/ (directory with classes)
    * *class_name*  
        * *recording in WAV format*  
        ...  
    ...

All classes are defined in `classes` variable below.

## Reading and parsing

In [1]:
import os
import os.path
import tempfile

REC_ORIGINAL_DIR = os.path.join(".", "recordings")
FILE_FORMAT = "wav"
SAMPLING_RATE = 44100.

DESCRIPTORS = [
    "attackstartendtimes",
    "logattacktime",
    "rms",
    "spectralcentroid",
    "spectralcrest",
    "spectralflatness",
    "spectralflux",
    "spectralkurtosis",
    "spectralrolloff",
    "spectralskewness",
    "spectralspread",
    "temporalcentroid",
    "zerocrossingrate",
]

temp_dir = tempfile.mkdtemp()
print("Using temporary directory: %s" % temp_dir)

Using temporary directory: /var/folders/6x/s90tdpss7plbh_387wf4vs7c0000gn/T/tmpQjiEam


In [2]:
# UTILITY FUNCTIONS

import subprocess
import errno
import csv

def create_path(path):
    """Creates a directory structure if it doesn't exist yet."""
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            sys.exit("Failed to create directory structure %s. Error: %s" % (path, exception))
            
def shellquote(s):
    return "'" + s.replace("'", "'\\''") + "'"

def sonic_annotator(descriptors, file_path, output_path):
    annotator_command = [
        "./sonic-annotator",
        "-t %s" % descriptors,
        shellquote(file_path),
        "-w csv",
        "--csv-basedir %s" % output_path,
        "--force",
        "--csv-force",
    ]
    subprocess.check_call(" ".join(annotator_command), shell=True)


def get_descriptor(directory, sound_name, descr_name):
    csv_name = "%s_vamp_mir-edu_%s_%s.csv" % (sound_name, descr_name, descr_name)
    with open(os.path.join(directory, csv_name), 'rb') as csv_file:
        values = []
        for row in csv.reader(csv_file):
            values.append([float(val) for val in row])
    return values

In [3]:
def get_recordings_in_class(cls_name):
    cls_dir = os.path.join(REC_ORIGINAL_DIR, cls_name)
    return [f[:-(len(FILE_FORMAT) + 1)] for f in os.listdir(cls_dir)
            if os.path.isfile(os.path.join(cls_dir, f)) and f.endswith(".%s" % FILE_FORMAT)]

classes = {
    "aggressive": get_recordings_in_class("aggressive"),
    "happy": get_recordings_in_class("happy"),
    "sad": get_recordings_in_class("sad"),
}

## Removing silences

In [4]:
import essentia.standard

SILENCE_THRESHOLD = 0.007  # RMS value

def ess_load_file(path):
    return essentia.standard.MonoLoader(filename=path)()

def get_ranges_below(value, data):
    """
    Args:
        value: Threshold value.
        data: List of pairs (time, value).
        
    Returns:
        List of tuples (start time, end time).
    """
    ranges = []  # list of tuples (start time, end time)
    current_start = None
    for time, value in data:
        if value < SILENCE_THRESHOLD:
            if current_start is None:
                current_start = time
        else:
            if current_start is not None:
                ranges.append((current_start, time))
                current_start = None
    if current_start is not None:
        ranges.append((current_start, data[-1][0]))
        current_start = None
    return ranges

def in_ranges(value, ranges):
    for r in ranges:
        if value >= r[0] and value <= r[1]:
            return True
    return False

def remove_ranges(recording, ranges):
    new_rec = []
    for sample_n, val in enumerate(recording):
        if not in_ranges(sample_n/SAMPLING_RATE, ranges):
            new_rec.append(val)
    return new_rec

In [5]:
import shutil

REC_CLEAN_DIR = os.path.join(temp_dir, "clean_recordings")

for cls_name, recordings in classes.iteritems():
    for rec_name in recordings:
        file_name = "%s.%s" % (rec_name, FILE_FORMAT)
        file_path = os.path.join(REC_ORIGINAL_DIR, cls_name, file_name)
        
        # Getting RMS descriptor using Sonic Annotator
        output_dir = os.path.join(temp_dir, "rms_annotation", cls_name)   
        create_path(output_dir)
        sonic_annotator("miredu_rms.n3", file_path, output_dir)
        rms = get_descriptor(output_dir, rec_name, "rms")

        # Removing silences using Essentia
        content = ess_load_file(file_path)
        content = remove_ranges(
            recording=content,
            ranges=get_ranges_below(SILENCE_THRESHOLD, rms)
        )
        
        print("Writing cleaned up %s/%s..." % (cls_name, file_name))
        output_dir = os.path.join(REC_CLEAN_DIR, cls_name)
        create_path(output_dir)
        essentia.standard.MonoWriter(
            filename=os.path.join(output_dir, file_name),
            format=FILE_FORMAT,
            sampleRate=SAMPLING_RATE,
        )(essentia.array(content))

print("Done!")

Writing cleaned up aggressive/07_Thillana_clip.wav...
Writing cleaned up aggressive/2Pac - Hit 'em Up.wav...
Writing cleaned up aggressive/Charlie Parker - Crazeology.wav...
Writing cleaned up aggressive/D12 - Fight Music.wav...
Writing cleaned up aggressive/Da Weasel - GTA.wav...
Writing cleaned up aggressive/Da Weasel - Tás Na Boa.wav...
Writing cleaned up aggressive/Diabolic feat. Immortal Technique - Frontlines.wav...
Writing cleaned up aggressive/DMX - Bring Your Whole Crew.wav...
Writing cleaned up aggressive/Doris Troy - Kill Them All!.wav...
Writing cleaned up aggressive/Earl Hines - Rock and rye.wav...
Writing cleaned up aggressive/Ella Fitzgerald - It don't mean a thing.wav...
Writing cleaned up aggressive/Eminem - Berzerk.wav...
Writing cleaned up aggressive/Eminem - Loose Yourself.wav...
Writing cleaned up aggressive/Goondox - Raps Of The Titans ft Swollen Members, Jus Allah, Virtuoso, Psych Ward, Jaysaun.wav...
Writing cleaned up aggressive/Grand Daddy IU - We Got Da Gats

## Computing descriptors

In [6]:
DESCRIPTORS_DIR = os.path.join(temp_dir, "annotator_output")

descriptors = {}

for cls_name, recordings in classes.iteritems():
    descriptors[cls_name] = {}
    
    for rec_name in recordings:
        descriptors[cls_name][rec_name] = {}
        
        file_name = "%s.%s" % (rec_name, FILE_FORMAT)
        file_path = os.path.join(REC_CLEAN_DIR, cls_name, file_name)
        output_path = os.path.join(DESCRIPTORS_DIR, cls_name)
        create_path(output_path)
        sonic_annotator("miredu_all.n3", file_path, output_path)
        
        for descriptor in DESCRIPTORS:
            descriptors[cls_name][rec_name][descriptor] = get_descriptor(output_path, rec_name, "rms")

## Generating ARFF file

Info about the format: **http://www.cs.waikato.ac.nz/~ml/weka/arff.html**

Using [liac-arff](https://pypi.python.org/pypi/liac-arff) package to generate the file.

Using the following functions to compute values:
* https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.std.html
* https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.mean.html

In [7]:
import numpy as np

attributes = []
for descr_name in DESCRIPTORS:
    # Order is important!
    attributes.append((descr_name + "_std", "REAL"))
    attributes.append((descr_name + "_mean", "REAL"))
attributes.append((u"mood", classes.keys()))

data = []
for cls_name, recordings in descriptors.iteritems():
    for rec_name, descr in recordings.iteritems():
        data_row = []
        for descr_name in DESCRIPTORS:
            # Order is important!
            values = [v[1] for v in descr[descr_name]]
            data_row.append(np.std(values))
            data_row.append(np.mean(values))
        data_row.append(cls_name)
        data.append(data_row)

dataset = {
    u"relation": u"Mood",
    u"attributes": attributes,
    u"data": data
}

In [8]:
import arff

ARFF_OUTPUT = "mood.arff"
with open(ARFF_OUTPUT, "w") as f:
    arff.dump(dataset, f)

/usr/local/lib/python2.7/site-packages/arff.py:366: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if value is None or value == u'' or value != value:


In [9]:
shutil.rmtree(temp_dir)  # Cleanup